<a href="https://colab.research.google.com/github/datakind/hxl-metadata-prediction/blob/main/openai-hxl-prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

A data standard on platforms such as the [Humanitarian Data Exchange (HDX)](https://data.humdata.org/) is the [Humanitarian Exchange Language (HXL)](https://hxlstandard.org/), a column level set of attributes and tags and attributes which improve data interoperability and discovery. These tags and attributes are typically set by hand by data owners, which being a manual process can result in poor dataset coverage. Improving coverage through ML and AI techniques is desirable for faster and more efficient use of data in responding to Humanitarian disasters.

Previous work has focussed on fine tuning LLMs to complete tags and attrubutes, starting with the study [Predicting Metadata on Humanitarian Datasets with GPT 3](https://medium.com/towards-data-science/predicting-metadata-for-humanitarian-datasets-using-gpt-3-b104be17716d). This has yielded promosing results, but is constrained by the quality of training data and the HDX team have confirmed that basic tags related to location and dates are popular, more esoteric tags defined in [the standard](https://hxlstandard.org/standard/1-1final/tagging/) are not well represented.

This notebook fine-tunes an OpenAI model to test performance.

# Setup

1. Run notebook [generate-test-train-data.ipynb]([generate-test-train-data.ipynb]) to generate test and train data files for use in fine-tuning
2. Set `OPENAI_API_KEY` in file `.env` or in Colab secrets

In [ ]:
!pip install pandas==2.2.2
!pip install openai==1.35.3
!pip install python-dotenv==1.0.1

In [14]:
import openai
import os
import time
import openai
from openai import OpenAI

from dotenv import load_dotenv
load_dotenv()

if os.getenv("OPENAI_API_KEY") is None:
  from google.colab import userdata
  OPENAI_API_KEY =  userdata.get('OPENAI_API_KEY')
else:
  OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# If using Colab, this is where Google drive gets mounted. Otherwise leave blank
GOOGLE_BASE_DIR = "/content/drive/MyDrive/Colab"

# Where to save local data files
LOCAL_DATA_DIR = f"{GOOGLE_BASE_DIR}/hxl-metadata-prediction/data/"

# As generated by generate-test-train-data.ipynb
TRAINING_FILE = f"{LOCAL_DATA_DIR}/hxl_chat_prompts.jsonl"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Analysis

## Fine-tune using Open AI

In [18]:
def fine_tune_model(train_file, model_name="gpt-4o-mini"):
    """
    Fine-tune an OpenAI model using training data.

    Args:
        prompt_file (str): The file containing the prompts to use for fine-tuning.
        model_name (str): The name of the model to fine-tune. Default is "davinci-002".

    Returns:
        str: The ID of the fine-tuned model.
    """

    client = OpenAI(
        api_key=OPENAI_API_KEY
    )

    # Create a file on OpenAI for fine-tuning
    file = client.files.create(
        file=open(train_file, "rb"),
        purpose="fine-tune"
    )
    file_id = file.id
    print(f"Uploaded training file with ID: {file_id}")

    # Start the fine-tuning job
    ft = client.fine_tuning.jobs.create(
        training_file=file_id,
        model=model_name
    )
    ft_id = ft.id
    print(f"Fine-tuning job started with ID: {ft_id}")

    # Monitor the status of the fine-tuning job
    ft_result = client.fine_tuning.jobs.retrieve(ft_id)
    while ft_result.status != 'succeeded':
        print(f"Current status: {ft_result.status}")
        time.sleep(120)  # Wait for 60 seconds before checking again
        ft_result = client.fine_tuning.jobs.retrieve(ft_id)
        if 'failed' in ft_result.status.lower():
            sys.exit()

    print(f"Fine-tuning job {ft_id} succeeded!")

    # Retrieve the fine-tuned model
    fine_tuned_model = ft_result.fine_tuned_model
    print(f"Fine-tuned model: {fine_tuned_model}")

    return fine_tuned_model

In [19]:
fine_tune_model(TRAINING_FILE, model_name="gpt-4o-mini-2024-07-18-alpha")

Uploaded training file with ID: file-CEZdeKIIj7PoQ7L8FePyWtvI
Fine-tuning job started with ID: ftjob-ULboB88nzvwFO25bqgIF6QvB
Current status: validating_files
Current status: validating_files
Current status: queued


KeyboardInterrupt: 

Mounted at /content/drive
